# 1. 2023年10月6日开始重构

- 1.akshare有一个接口更新了名字
- 2.券商研报的表头更新了
- 3.通过萝卜投研爬取10年pe平均值做了线程池优化（12个workers让本来50分钟的爬取减少到5分钟）

# 2024年7月3日更新券商研报的表头，其他都没变化
# 2024年8月19日更新akshare=1.14.62 接口行业的表头

# 2025年1月1日重构 因legulegu.com的接口对爬虫效率进行了封禁，封半小时。 因此随机化sleep，可以爬了

In [1]:
import akshare as ak
# (base) [1,2,3,9,10,11,12]
ak.stock_profit_forecast_em().head(5)

  0%|          | 0/6 [00:00<?, ?it/s]

,序号,代码,名称,研报数,机构投资评级(近六个月)-买入,机构投资评级(近六个月)-增持,机构投资评级(近六个月)-中性,机构投资评级(近六个月)-减持,机构投资评级(近六个月)-卖出,2023预测每股收益,2024预测每股收益,2025预测每股收益,2026预测每股收益
0,1,600519,贵州茅台,54,47.0,7.0,0.0,0.0,0.0,59.492280,68.552926,76.680037,85.386075
1,2,000858,五粮液,50,40.0,10.0,0.0,0.0,0.0,7.783008,8.487680,9.232860,10.125469
2,3,603605,珀莱雅,49,37.0,12.0,0.0,0.0,0.0,3.012935,3.876612,4.800878,5.798563
3,4,300750,宁德时代,47,36.0,11.0,0.0,0.0,0.0,10.019663,11.665891,14.195370,16.803422
4,5,600809,山西汾酒,47,38.0,9.0,0.0,0.0,0.0,8.556082,10.244915,11.825532,13.615152


In [2]:
#!/usr/bin/env python=3.10
# coding=utf-8
# author: KarlQu
# Version: 1.3
# Date: 2025-01-01
# webdriver=Firefox2023.04
# pip install akshare -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com  --upgrade
#  mamba install -c conda-forge selenium=4.13.0
# selenium 4 与 selenium 3 对对象选择语法做了修改
#  mamba install -c conda-forge jupyter
# akshare.__version__ = '1.14.62 '
# !最重要的是g和g的可达性
# akshare更新非常频繁
# pip install akshare --upgrade -i https://pypi.org/simple

# 是否需要更新10年pe均值和券商研报, pe_ave_10(ifupdate = False), 输出位置 f'./backups/{DATE}/nets_end.xlsx'

import numpy as np
import pandas as pd
import requests
import akshare as ak
import tqdm
import time
import datetime as dt
import os
import calendar
from selenium import webdriver
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
from concurrent import futures

DATE = dt.date.today() # str(DATE) ; dt.date.today().year

UA = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'
}

def open_file(file_name:str, upperdir = 'backups'):
    """创建文件夹
    Parameters
    ----------
    file_name : 日期字符串
    upperdir : 上级目录名
    """
    if os.path.exists(f'./{upperdir}/' + file_name):
        pass
    else:
        os.mkdir(f'./{upperdir}/' + file_name)

def update_g(ifupdate = False) -> pd.DataFrame:
    """akshare的个股研报接口

    Parameters
    ----------
    ifupdate : bool, optional
        _description_, by default False

    Returns
    -------
    pd.DataFrame
        columns = ['代码','名称','研报数','2021预测每股收益','2022预测每股收益','2023预测每股收益','2024预测每股收益']
        
        共4年的每股收益,2023年更新为'2022预测每股收益','2023预测每股收益','2024预测每股收益','2025预测每股收益'

    """
    if ifupdate:
        open_file(str(DATE), upperdir = 'backups')
        profit_forecast = ak.stock_profit_forecast_em().iloc[:,[1,2,3,9,10,11,12]].drop_duplicates()
        profit_forecast.to_excel(f'./backups/{DATE}/profit_forecast.xlsx',index=False)

    else:
        latest_file = list(os.walk('./backups/'))[0][1][-1]
        profit_forecast = pd.read_excel(f'./backups/{latest_file}/profit_forecast.xlsx',dtype={'代码':'str'})
    # 调整g,收益>0、每期g>10%、每期g可信值35%。 得到调整的几何均数g
    profit_forecast = profit_forecast[(profit_forecast['2023预测每股收益']>0) & (profit_forecast['2024预测每股收益']>0 )& ( profit_forecast['2025预测每股收益']>0)]
    profit_forecast
    profit_forecast1 = profit_forecast.iloc[:,[3,4,5,6]].pct_change(axis=1)[['2024预测每股收益','2025预测每股收益',	'2026预测每股收益']]
    profit_forecast1
    profit_forecast2 = profit_forecast1[(profit_forecast1['2024预测每股收益']>0.1) & (profit_forecast1['2025预测每股收益']>0.1 )& ( profit_forecast1['2026预测每股收益']>0.1)]
    profit_forecast2
    profit_forecast2[profit_forecast2>0.35] = 0.35
    profit_forecast3 = profit_forecast2 + 1
    g_scaled = ((profit_forecast3['2024预测每股收益']*profit_forecast3['2025预测每股收益']*profit_forecast3['2026预测每股收益'])**(1/3)-1)
    g_scaled.name = 'g'
    g = pd.concat([g_scaled,profit_forecast],axis=1,join='inner')[['代码','名称','g','研报数']].reset_index(drop=True)
    return g


def ten_year_ago(date: dt.date = DATE) -> str:
        """10年前日期，月底对应月底
        Parameters
        ----------
        date : datetime.date
        Returns
        -------
        str
        """
        year = date.year - 10
        day = min(date.day, calendar.monthrange(year, date.month)[1])
        dt = date.replace(year=year, month=date.month, day=day)
        # print(dt)
        bigen_date = str(dt).replace('-','')
        return bigen_date

# 是否需要更新10年pe均值和券商研报, 默认pe_ave_10(ifupdate = False), 
# 输出位置 f'./backups/{DATE}/nets_end.xlsx'
def pe_ave_10(date:str = str(DATE), ifupdate = False):
    """获取or更新10年pe均值和券商研报的预期每股收益

    Parameters
    ----------
    date : str, optional
        _description_, by default str(DATE)
    ifupdate : bool, optional
        _description_, by default False
    """
    if ifupdate:
        g = update_g(ifupdate=ifupdate)
        stock_ids = g.代码.to_list() # 每次爬pe历史均值的时候都更新一下券商研报
        print('代码前五个')
        print(stock_ids[0:5])
        print('-----------------开始爬取萝卜pe历史均值---------------')
        # 1. selenium登录获取cookies
        denglu_link = 'https://robo.datayes.com/v2/landing/peband'
        cruser = webdriver.Firefox()
        cruser.get(denglu_link)
        input("扫码登录后按回车键")
        # WebDriverWait(cruser,60).until(EC.presence_of_all_elements_located((By.ID, 'app')))
        cookies = {x['name']:x['value'] for x in cruser.get_cookies()}
        print(cookies)
        end_date = date.replace('-','')
        begin_date = ten_year_ago()
        ave_pettm = []
        executor = futures.ThreadPoolExecutor(max_workers=12)
        fs = []
        total_tasks = len(stock_ids)
        with tqdm.tqdm(total=total_tasks) as pbar:
            for ids in stock_ids:
                def get_ids_peave(ids, begin_date, end_date, cookies, UA, ave_pettm):
                    try:
                        url = f'https://gw.datayes.com/rrp_adventure/web/stockModel/band/{ids}' # 2022old
                        response = requests.get(url=url,
                                                params= {
                                                    'apiType': '4',
                                                    'category': '1',    # 1为PE，2为PB
                                                    'subCategory': '1',
                                                    'flag': '-1',
                                                    'beginDate': f'{begin_date}',
                                                    'endDate': f'{end_date}'
                                                },
                                                cookies=cookies,
                                                headers=UA)
                        ave_pettm.append([ids, response.json()['data']['mean']])
                        pbar.update(1) # 更新tqdm进度条用
                        time.sleep(np.random.rand()) # 1536/1536 [33:05<00:00,  1.29s/it]
                        # print(f'正常{response.status_code} , code: {ids}')
                    except:
                        print(f'异常{response.status_code} , code: {ids}')
                        pass

                f= executor.submit(get_ids_peave, ids, begin_date, end_date, cookies, UA, ave_pettm)
                fs.append(f)
            futures.wait(fs)
        cruser.quit()
        # 3. 本地保存
        pe_ave = pd.DataFrame(ave_pettm, columns=['code','ave_pettm'])
        open_file(str(DATE), upperdir = 'backups')
        pe_ave.to_excel(f'./backups/{DATE}/avepettm_backup.xlsx',index=False)
        latest_file = DATE
    else:
        latest_file = list(os.walk('./backups/'))[0][1][-1]
        print(f'研报和10年期pe平均值的最近一次更新时间为 {latest_file}')
        g = update_g(ifupdate=ifupdate)
        pe_ave = pd.read_excel(f'./backups/{latest_file}/avepettm_backup.xlsx',dtype={'code':'str'})
    nets = pd.merge(g.rename(columns={'代码':'code'}) , pe_ave , on='code',how='left')
    nets['peg'] = (nets.ave_pettm / (100*nets.g))
    nets1 = nets[(nets['peg']<1.2)]
    # (nets['peg']<1.2).sum()
    # 2. 得到筛选后的股票代码，后续可以爬取pettm和dvttm了
    stock_ids_selected = nets1.code.to_list()
    print('DEBUG:print 代码前五')
    print(stock_ids_selected[0:5])
    pe_ttm = []
    for ids in tqdm.tqdm(stock_ids_selected):

        try:
            rrr = ak.stock_a_indicator_lg(symbol=f'{ids}').iloc[-1,[0,2,7]] # pettm and dvttm 接口
            # print(f"rrr {rrr}")
            rrr['industry'] = ak.stock_individual_info_em(symbol=f"{ids}").loc[6].value # 行业接口
            # print(f"rrr with industry {rrr}")
            rrr.name = ids
            pe_ttm.append(rrr)

            time.sleep(1 + np.random.rand()) # 反扒试试。。
            
        except:
            print(f'{ids} 查无!')
        continue

    pe_and_dv = pd.concat(pe_ttm,axis=1).T
    pe_and_dv_backup = pe_and_dv.reset_index(drop=False)
    pe_and_dv_backup = pe_and_dv_backup.rename(columns={'index':'code'}) 
    open_file(str(DATE), upperdir = f'backups/{latest_file}')
    pe_and_dv_backup.to_excel(f'./backups/{latest_file}/{DATE}/pe_and_dv.xlsx',index=False)
    # pe_ttm = pd.read_excel(f'./backups/{latest_file}/{DATE}/pe_and_dv.xlsx',dtype={'code':'str'})
    nets2 = pd.merge(nets1,pe_and_dv_backup,on='code',how='left')
    nets3 = nets2[nets2.pe_ttm<nets2.ave_pettm]
    nets3['peg'] = (nets3.pe_ttm / (100*nets3.g))
    nets3['exp_reward_yearly'] = 100*((nets3.ave_pettm/nets3.pe_ttm)**(1/3)*(1+nets3.g)-1)
    nets_end = nets3[nets3.exp_reward_yearly>35].drop_duplicates()
    nets_end.to_excel(f'./backups/{latest_file}/{DATE}/nets_end.xlsx',index=False)   
    print('done! open nets_end.xlsx')

## 行业接口的表头由[2]到[6]

In [3]:
ak.stock_individual_info_em(symbol=f"603198").loc[6].value

'酿酒行业'

In [28]:
# 网站对爬虫速度进行了限制
ak.stock_a_indicator_lg(symbol=f'603198').iloc[-1,[0,2,7]] # pettm and dvttm 接口


# 如果上面不行试试更新akshare版本
# !pip install akshare --upgrade -i https://pypi.org/simple

trade_date    2024-12-31
pe_ttm           16.3509
dv_ttm            2.4101
Name: 2334, dtype: object

In [17]:
# 从akshare里截取的实现

from datetime import datetime
from hashlib import md5

import pandas as pd
import requests
from bs4 import BeautifulSoup

from akshare.utils.cons import headers

def get_cookie_csrf(url: str = "") -> dict:
    """
    乐咕乐股-主板市盈率
    https://legulegu.com/stockdata/shanghaiPE
    :return: 指定市场的市盈率数据
    :rtype: pandas.DataFrame
    """
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    csrf_tag = soup.find(name="meta", attrs={"name": "_csrf"})
    csrf_token = csrf_tag.attrs["content"]
    headers.update({"X-CSRF-Token": csrf_token})
    return {"cookies": r.cookies, "headers": headers}


def get_token_lg() -> str:
    """
    生成乐咕的 token
    https://legulegu.com/s/002488
    :return: token
    :rtype: str
    """
    current_date_str = datetime.now().date().isoformat()
    obj = md5()
    obj.update(current_date_str.encode("utf-8"))
    token = obj.hexdigest()
    return token

symbol = "600761"
url = "https://legulegu.com/api/s/base-info/"
token = get_token_lg()
params = {"token": token, "id": symbol}
r = requests.post(
    url,
    params=params,
    **get_cookie_csrf(url="https://legulegu.com/"),
)
temp_json = r.json()
temp_df = pd.DataFrame(
    temp_json["data"]["items"],
    columns=temp_json["data"]["fields"],
)
print(temp_df)
temp_df["trade_date"] = pd.to_datetime(temp_df["trade_date"]).dt.date
temp_df[temp_df.columns[1:]] = temp_df[temp_df.columns[1:]].astype(float)
temp_df.sort_values(by=["trade_date"], inplace=True, ignore_index=True)
if len(set(temp_df["trade_date"])) <= 0:
    raise ValueError("数据获取失败, 请检查是否输入正确的股票代码")

     trade_date       pe   pe_ttm      pb      ps  ps_ttm dv_ratio  dv_ttm  \
0      20241231  12.2998  11.2867  1.5371  0.8998  0.8858   1.8833  3.0122   
1      20241230  12.7179  11.6704  1.5893  0.9304  0.9159   1.8214  2.9132   
2      20241227  12.6761   11.632  1.5841  0.9274  0.9129   1.8274  2.9228   
3      20241226  12.7598  11.7088  1.5946  0.9335  0.9189   1.8154  2.9036   
4      20241225  12.5786  11.5425  1.5719  0.9202  0.9059   1.8416  2.9454   
...         ...      ...      ...     ...     ...     ...      ...     ...   
5595   20010928  33.0156  25.8967  2.8183  3.9697  3.6028      0.0  6.9242   
5596   20010927  32.4191  25.4289  2.7674   3.898  3.5377      0.0  7.0516   
5597   20010926  32.3139  25.3463  2.7584  3.8854  3.5262      0.0  7.0746   
5598   20010925  31.7174  24.8785  2.7075  3.8136  3.4611      0.0  7.2076   
5599   20010924  30.9455   24.273  2.6416  3.7208  3.3769      0.0  7.3874   

          total_mv  
0     1572072.7232  
1     1625514.2888  


In [ ]:
# 真正的运行吧！

# pe_ave_10(ifupdate=True)
pe_ave_10(ifupdate=False)

研报和10年期pe平均值的最近一次更新时间为 2025-01-01


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14988\2610632021.py:79: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  profit_forecast1 = profit_forecast.iloc[:,[3,4,5,6]].pct_change(axis=1)[['2024预测每股收益','2025预测每股收益',	'2026预测每股收益']]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14988\2610632021.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_forecast2[profit_forecast2>0.35] = 0.35
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14988\2610632021.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

DEBUG:print 代码前五
['600761', '603193', '002648', '600426', '003006']


100%|██████████| 264/264 [08:45<00:00,  1.99s/it]

done! open nets_end.xlsx



C:\Users\Administrator\AppData\Local\Temp\ipykernel_14988\2610632021.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nets3['peg'] = (nets3.pe_ttm / (100*nets3.g))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14988\2610632021.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nets3['exp_reward_yearly'] = 100*((nets3.ave_pettm/nets3.pe_ttm)**(1/3)*(1+nets3.g)-1)


# 后记
## 1.使用结果net_end.xlsx来选股
> tips:2022-10-14选的股，到现在(2024.7.3)看都有了35%的收益

**net_end.xlsx**

- 1 首先用peg和pe_ttm做降序排序
- 2 看pe_ttm与10年平均pe的距离
- 3 优先选dv_ttm（股息率）高于3.4的
- 4 对比g（券商研报）,看能不能支撑pe_ttm的回报率

## 2.pip install -r requirements.txt:


> akshare==1.15.61
anyio @ file:///C:/b/abs_847uobe7ea/croot/anyio_1706220224037/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci/argon2-cffi-bindings_1644569876605/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/async-lru_1699554572212/work
attrs @ file:///C:/b/abs_35n0jusce8/croot/attrs_1695717880170/work
Babel @ file:///C:/b/abs_a2shv_3tqi/croot/babel_1671782804377/work
backcall==0.2.0
beautifulsoup4 @ file:///C:/b/abs_d5wytg_p0w/croot/beautifulsoup4-split_1718029833749/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
Brotli @ file:///C:/b/abs_3d36mno480/croot/brotli-split_1714483178642/work
certifi @ file:///C:/b/abs_1fw_exq1si/croot/certifi_1725551736618/work/certifi
cffi @ file:///C:/b/abs_78eb1_vq6z/croot/cffi_1714483206096/work
charset-normalizer @ file:///croot/charset-normalizer_1721748349566/work
colorama @ file:///C:/b/abs_a9ozq0l032/croot/colorama_1672387194846/work
comm @ file:///C:/b/abs_67a8058udb/croot/comm_1709322909844/work
contourpy @ file:///D:/bld/contourpy_1725378323525/work
cycler @ file:///tmp/build/80754af9/cycler_1637851556182/work
debugpy @ file:///C:/b/abs_c0y1fjipt2/croot/debugpy_1690906864587/work
decorator @ file:///opt/conda/conda-bld/decorator_1643638310831/work
defusedxml @ file:///tmp/build/80754af9/defusedxml_1615228127516/work
docopt==0.6.2
et-xmlfile==1.1.0
exceptiongroup @ file:///C:/b/abs_c5h1o1_b5b/croot/exceptiongroup_1706031441653/work
executing @ file:///opt/conda/conda-bld/executing_1646925071911/work
fastjsonschema @ file:///C:/Users/BUILDE~1/AppData/Local/Temp/abs_ebruxzvd08/croots/recipe/python-fastjsonschema_1661376484940/work
fonttools @ file:///C:/b/abs_f47gnfqnx0/croot/fonttools_1713551644747/work
h11 @ file:///C:/b/abs_1czwoyexjf/croot/h11_1706652332846/work
html5lib==1.1
idna @ file:///C:/b/abs_aad84bnnw5/croot/idna_1714398896795/work
ipykernel @ file:///C:/b/abs_c2u94kxcy6/croot/ipykernel_1705933907920/work
ipython==8.12.3
ipywidgets @ file:///C:/b/abs_25rsd7yhd4/croot/ipywidgets_1709575117021/work
jedi @ file:///C:/b/abs_1b8kmj7rrm/croot/jedi_1721058359741/work
Jinja2 @ file:///C:/b/abs_92fccttino/croot/jinja2_1716993447201/work
json5 @ file:///tmp/build/80754af9/json5_1624432770122/work
jsonpath==0.82.2
jsonschema @ file:///C:/b/abs_d1c4sm8drk/croot/jsonschema_1699041668863/work
jsonschema-specifications @ file:///C:/b/abs_0brvm6vryw/croot/jsonschema-specifications_1699032417323/work
jupyter @ file:///C:/b/abs_4e102rc6e5/croot/jupyter_1707947170513/work
jupyter-console @ file:///C:/b/abs_82xaa6i2y4/croot/jupyter_console_1680000189372/work
jupyter-events @ file:///C:/b/abs_c2m9s5b5m5/croot/jupyter_events_1718738115254/work
jupyter-lsp @ file:///C:/b/abs_ecle3em9d4/croot/jupyter-lsp-meta_1699978291372/work
jupyter_client @ file:///C:/b/abs_a6h3c8hfdq/croot/jupyter_client_1699455939372/work
jupyter_core @ file:///C:/b/abs_beftpbuevw/croot/jupyter_core_1718818307097/work
jupyter_server @ file:///C:/b/abs_9a333nh6yu/croot/jupyter_server_1718827092223/work
jupyter_server_terminals @ file:///C:/b/abs_ec0dq4b50j/croot/jupyter_server_terminals_1686870763512/work
jupyterlab @ file:///C:/b/abs_43venm28fu/croot/jupyterlab_1706802651134/work
jupyterlab-pygments @ file:///tmp/build/80754af9/jupyterlab_pygments_1601490720602/work
jupyterlab-widgets @ file:///C:/b/abs_62picw9xos/croot/jupyterlab_widgets_1709323131962/work
jupyterlab_server @ file:///C:/b/abs_e08i7qn9m8/croot/jupyterlab_server_1699555481806/work
kiwisolver @ file:///C:/b/abs_88mdhvtahm/croot/kiwisolver_1672387921783/work
lxml==5.3.0
MarkupSafe @ file:///C:/b/abs_ecfdqh67b_/croot/markupsafe_1704206030535/work
matplotlib==3.9.2
matplotlib-inline @ file:///C:/ci/matplotlib-inline_1661934094726/work
mini-racer==0.12.4
mistune @ file:///C:/Users/BUILDE~1/AppData/Local/Temp/abs_081kimkskf/croots/recipe/mistune_1661496225923/work
mkl-service==2.4.0
mkl_fft @ file:///C:/b/abs_f55mv94vyg/croot/mkl_fft_1725370278455/work
mkl_random @ file:///C:/b/abs_21ydbzdu8d/croot/mkl_random_1725370276095/work
nbclient @ file:///C:/b/abs_cal0q5fyju/croot/nbclient_1698934263135/work
nbconvert==7.16.4
nbformat @ file:///C:/b/abs_5a2nea1iu2/croot/nbformat_1694616866197/work
nest-asyncio @ file:///C:/b/abs_65d6lblmoi/croot/nest-asyncio_1708532721305/work
notebook @ file:///C:/b/abs_09nvambyty/croot/notebook_1719499424678/work
notebook_shim @ file:///C:/b/abs_a5xysln3lb/croot/notebook-shim_1699455926920/work
numpy @ file:///C:/b/abs_0123vcxhf8/croot/numpy_and_numpy_base_1725470331966/work/dist/numpy-2.0.1-cp310-cp310-win_amd64.whl#sha256=e824ec3b279f4c5207736c83ac2f2dc79b489e08501a4ea9e3fc178c45db289b
openpyxl @ file:///C:/b/abs_0e6ca21lac/croot/openpyxl_1721752965859/work
outcome @ file:///tmp/build/80754af9/outcome_1609338780791/work
overrides @ file:///C:/b/abs_cfh89c8yf4/croot/overrides_1699371165349/work
packaging @ file:///C:/b/abs_c3vlh0z4jw/croot/packaging_1720101866539/work
pandas @ file:///D:/bld/pandas_1715897730709/work
pandocfilters @ file:///opt/conda/conda-bld/pandocfilters_1643405455980/work
parso @ file:///opt/conda/conda-bld/parso_1641458642106/work
patsy @ file:///home/conda/feedstock_root/build_artifacts/patsy_1704469236901/work
pickleshare==0.7.5
pillow @ file:///C:/b/abs_32o8er3uqp/croot/pillow_1721059447598/work
pipreqs==0.5.0
platformdirs @ file:///C:/b/abs_b6z_yqw_ii/croot/platformdirs_1692205479426/work
plotly @ file:///C:/b/abs_9asyl7560v/croot/plotly_1718136949929/work
plotly-express==0.4.1
ply==3.11
prometheus-client @ file:///C:/Windows/TEMP/abs_ab9nx8qb08/croots/recipe/prometheus_client_1659455104602/work
prompt-toolkit @ file:///C:/b/abs_68uwr58ed1/croot/prompt-toolkit_1704404394082/work
psutil @ file:///C:/Windows/Temp/abs_b2c2fd7f-9fd5-4756-95ea-8aed74d0039flsd9qufz/croots/recipe/psutil_1656431277748/work
pure-eval @ file:///opt/conda/conda-bld/pure_eval_1646925070566/work
pycparser @ file:///tmp/build/80754af9/pycparser_1636541352034/work
Pygments @ file:///C:/b/abs_fay9dpq4n_/croot/pygments_1684279990574/work
pyparsing @ file:///C:/b/abs_520r19rysg/croot/pyparsing_1725041656021/work
PyQt5==5.15.10
PyQt5-sip @ file:///C:/b/abs_c0pi2mimq3/croot/pyqt-split_1698769125270/work/pyqt_sip
PySocks @ file:///C:/ci_310/pysocks_1642089375450/work
python-dateutil @ file:///C:/b/abs_3au_koqnbs/croot/python-dateutil_1716495777160/work
python-json-logger @ file:///C:/b/abs_cblnsm6puj/croot/python-json-logger_1683824130469/work
pytz @ file:///C:/b/abs_6ap4tsz1ox/croot/pytz_1713974360290/work
pywin32==305.1
pywinpty @ file:///C:/b/abs_73vshmevwq/croot/pywinpty_1677609966356/work/target/wheels/pywinpty-2.0.10-cp310-none-win_amd64.whl
PyYAML @ file:///C:/b/abs_782o3mbw7z/croot/pyyaml_1698096085010/work
pyzmq @ file:///C:/b/abs_89aq69t0up/croot/pyzmq_1705605705281/work
qtconsole @ file:///C:/b/abs_03f8rg9vl6/croot/qtconsole_1709231218069/work
QtPy @ file:///C:/b/abs_derqu__3p8/croot/qtpy_1700144907661/work
referencing @ file:///C:/b/abs_09f4hj6adf/croot/referencing_1699012097448/work
requests @ file:///C:/b/abs_9frifg92q2/croot/requests_1721410901096/work
rfc3339-validator @ file:///C:/b/abs_ddfmseb_vm/croot/rfc3339-validator_1683077054906/work
rfc3986-validator @ file:///C:/b/abs_6e9azihr8o/croot/rfc3986-validator_1683059049737/work
rpds-py @ file:///C:/b/abs_76j4g4la23/croot/rpds-py_1698947348047/work
scipy @ file:///C:/bld/scipy-split_1724327194933/work/dist/scipy-1.14.1-cp310-cp310-win_amd64.whl#sha256=e6bdc831fca55b320340d9d65555e680654f474b6605e599cfd60b4c00f7d5d6
selenium @ file:///home/conda/feedstock_root/build_artifacts/selenium_1696010260379/work
Send2Trash @ file:///C:/b/abs_08dh49ew26/croot/send2trash_1699371173324/work
sip @ file:///C:/b/abs_edevan3fce/croot/sip_1698675983372/work
six @ file:///tmp/build/80754af9/six_1644875935023/work
sniffio @ file:///C:/b/abs_3akdewudo_/croot/sniffio_1705431337396/work
sortedcontainers @ file:///tmp/build/80754af9/sortedcontainers_1623949099177/work
soupsieve @ file:///C:/b/abs_bbsvy9t4pl/croot/soupsieve_1696347611357/work
stack-data @ file:///opt/conda/conda-bld/stack_data_1646927590127/work
statsmodels @ file:///D:/bld/statsmodels_1715941409651/work
tabulate==0.9.0
tenacity @ file:///C:/b/abs_d2_havtscu/croot/tenacity_1721222514290/work
terminado @ file:///C:/b/abs_25nakickad/croot/terminado_1671751845491/work
tinycss2 @ file:///C:/b/abs_52w5vfuaax/croot/tinycss2_1668168823131/work
tomli @ file:///C:/Windows/TEMP/abs_ac109f85-a7b3-4b4d-bcfd-52622eceddf0hy332ojo/croots/recipe/tomli_1657175513137/work
tornado @ file:///C:/b/abs_7bua0304mj/croot/tornado_1718740122405/work
tqdm==4.66.5
traitlets @ file:///C:/b/abs_bfsnoxl4pq/croot/traitlets_1718227069245/work
trio @ file:///C:/b/abs_3bsokxbl8q/croot/trio_1705518572139/work
trio-websocket @ file:///home/conda/feedstock_root/build_artifacts/trio-websocket_1695816857197/work/dist
typing_extensions @ file:///C:/b/abs_0as9mdbkfl/croot/typing_extensions_1715268906610/work
tzdata @ file:///croot/python-tzdata_1690578112552/work
unicodedata2 @ file:///C:/b/abs_b6apldlg7y/croot/unicodedata2_1713212998255/work
urllib3 @ file:///C:/b/abs_aen1x8xlvc/croot/urllib3_1688592717643/work
wcwidth @ file:///Users/ktietz/demo/mc3/conda-bld/wcwidth_1629357192024/work
webencodings==0.5.1
websocket-client @ file:///C:/b/abs_5dmnxxoci9/croot/websocket-client_1715878351319/work
widgetsnbextension @ file:///C:/b/abs_a84ycxud7_/croot/widgetsnbextension_1709322945280/work
win-inet-pton @ file:///C:/ci_310/win_inet_pton_1642658466512/work
wsproto @ file:///home/conda/feedstock_root/build_artifacts/wsproto_1661356345548/work
xlrd==2.0.1
yarg==0.1.9

In [3]:
import os
list(os.walk('./backups/')) # [0][1][-1]

[('./backups/', ['2022-10-16', '2023-10-06'], []),
 ('./backups/2022-10-16',
  [],
  ['avepettm_backup.xlsx',
   'nets_end.xlsx',
   'pe_and_dv.xlsx',
   'profit_forecast.xlsx']),
 ('./backups/2023-10-06',
  [],
  ['avepettm_backup.xlsx',
   'nets_end.xlsx',
   'pe_and_dv.xlsx',
   'profit_forecast.xlsx'])]